<a href="https://colab.research.google.com/github/yingzhi-w/geodata_linebot/blob/main/geodata_linebot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /drive
#umount /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p /drive/ngrok-ssh
!mkdir -p ~/.ssh

Mounted at /content/drive


In [2]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2022-06-22 18:45:50--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  3.56MB/s    in 5.0s    

2022-06-22 18:45:56 (2.63 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip


In [3]:
!/ngrok authtoken 2AAyO4ePNUakjl0gGLjcpWfzDI2_3xA1YgSgEp4rvG7dQZcib

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
!pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install line-bot-sdk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 4.5 MB/s 
     |████████████████████████████████| 1.1 MB 19.0 MB/s 
     |████████████████████████████████| 144 kB 42.8 MB/s 
     |████████████████████████████████| 271 kB 38.2 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 


In [8]:
!pip install mf2py
import mf2py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for mf2py: filename=mf2py-1.1.2-py3-none-any.whl size=23244 sha256=a3861a0456ace20c9bfab3751fa16ed333259db68fa120d3c091cc49b4d012ed
  Stored in directory: /root/.cache/pip/wheels/d9/47/72/d1f19d30e8dd2c06e7084d93ea90ab5ec60c6db488398c67db
Successfully built mf2py


In [9]:
def get_geo(doc):
  coords = []
  for d in doc['items']:
    try:
      data = {
          'name': d['properties']['name'][0],  #地名
          'geo': d['properties']['geo'][0]['value']  #座標
          }
      coords.append(data)
    except (IndexError, KeyError):
      pass
  return coords

In [13]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request

# 載入 LINE Message API 相關函式庫
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage

# 載入 json 標準函式庫，處理回傳的資料格式
import json

app = Flask(__name__)

@app.route("/", methods=['POST'])
def linebot():
    body = request.get_data(as_text=True)                   # 取得收到的訊息內容
    try:
        json_data = json.loads(body)                         # json 格式化訊息內容
        access_token = 'lQbD17cxSKLqvT2TpjqbEn0mYE2y7cFWwfn0In09SvyCBTBMNZwJuTGNUrokLGAQcCkiCO2juCwdCFpzBMvntDl1F04C2RaxaSADsx1BajNInJvGpcplAM9SMPXTz/k4ycZwSPialrD7la5glMlvtwdB04t89/1O/w1cDnyilFU='
        secret = '28e0afc7cab79c8c244e26ff4a7aad25'
        line_bot_api = LineBotApi(access_token)              # 確認 token 是否正確
        handler = WebhookHandler(secret)                     # 確認 secret 是否正確
        signature = request.headers['X-Line-Signature']      # 加入回傳的 headers
        handler.handle(body, signature)                      # 綁定訊息回傳的相關資訊
        msg = json_data['events'][0]['message']['text']      # 取得 LINE 收到的文字訊息
        tk = json_data['events'][0]['replyToken']            # 取得回傳訊息的 Token

        c = msg
        doc = mf2py.parse(url=f"https://en.wikipedia.org/wiki/{c}", html_parser='lxml')
        coords = get_geo(doc)
        for item in coords:
          geo_data=item["geo"].split("; ")
        line_bot_api.reply_message(tk,TextSendMessage(f"緯度: {geo_data[0]}\n經度: {geo_data[1]}"))  # 回傳訊息
        print(msg, tk)                                       # 印出內容
    except:
        print(body)                                          # 如果發生錯誤，印出收到的內容
    return 'OK' 
                    # 驗證 Webhook 使用，不能省略
if __name__ == "__main__":
  run_with_ngrok(app)           # 串連 ngrok 服務
  app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7439-34-125-45-247.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [22/Jun/2022 19:49:36] "POST / HTTP/1.1" 200 -


London 9c45649c08cc457dbf5b22cae105c781


...

London 70521af7680149b8a497521dcbaf861a


127.0.0.1 - - [22/Jun/2022 19:51:20] "POST / HTTP/1.1" 200 -


Paris 50b6b0c72afa46d681fde3b99edf4223


127.0.0.1 - - [22/Jun/2022 19:51:28] "POST / HTTP/1.1" 200 -


Taipei 4f33d65fa47d4b2281700debb2685452
